In [ ]:
pip install folium

In [ ]:
pip install geopandas

In [ ]:
pip install openpyxl


In [5]:
import geopandas as gpd
import folium

df = pd.read_csv(r"C:\Users\Administrator\Desktop\데분개\Geomap\최종생활권.csv")
gdf = gpd.read_file(r"C:\Users\Administrator\Desktop\데분개\Geomap\only생활권.geojson")

In [ ]:
print(gdf.head())
print(gdf.columns)

   ID          adm_nm     adm_cd2    sgg sido sidonm sggnm    adm_cd  \
0   1   서울특별시 종로구 사직동  1111053000  11110   11  서울특별시   종로구  11010530   
1   2   서울특별시 종로구 부암동  1111055000  11110   11  서울특별시   종로구  11010550   
2   3   서울특별시 종로구 무악동  1111057000  11110   11  서울특별시   종로구  11010570   
3   4   서울특별시 종로구 가회동  1111060000  11110   11  서울특별시   종로구  11010600   
4   5  서울특별시 종로구 창신1동  1111067000  11110   11  서울특별시   종로구  11010670   

                        시도              시군구                          읍면동  \
0        서울특별시;서울특별시;서울특별시      종로구;종로구;종로구                청운효자동;사직동;삼청동   
1              서울특별시;서울특별시          종로구;종로구                      부암동;평창동   
2              서울특별시;서울특별시          종로구;종로구                      무악동;교남동   
3  서울특별시;서울특별시;서울특별시;서울특별시  종로구;종로구;종로구;종로구  가회동;종로1.2.3.4가동;종로5.6가동;이화동   
4        서울특별시;서울특별시;서울특별시      종로구;종로구;종로구              혜화동;창신제1동;창신제2동   

       행정기관코드   총인구수   어린이    노인  \
0  1111053000  22112  2808  4824   
1  1111055000  26249  3568  5874   
2 

In [2]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# 1. 엑셀 불러오기
df = pd.read_excel(r"C:\Users\Administrator\Desktop\데분개\데이터\측정소 데이터\측정소별_정보및평균.xlsx")

# 2. 좌표를 숫자로 강제 변환 + NaN 제거
df['dmX'] = pd.to_numeric(df['dmX'], errors='coerce')  # 경도
df['dmY'] = pd.to_numeric(df['dmY'], errors='coerce')  # 위도
df = df.dropna(subset=['dmX', 'dmY'])  # NaN 제거

# 👉 여기서 꼭 확인! 숫자형으로 변환됐는지
print(df[['dmX', 'dmY']].dtypes)      # float64로 나와야 정상
print(df[['dmX', 'dmY']].head())      # 숫자처럼 보이는지 확인


dmX    float64
dmY    float64
dtype: object
         dmX         dmY
0  35.963042  129.376848
1  36.141641  128.438685
2  34.076677  125.092276
3  37.201739  127.545135
4  36.550344  127.550333


**생활권 데이터와 함께 활용하기 위해 geoJson 정보 넣기**

In [3]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# 1. 엑셀 파일 불러오기
df = pd.read_excel(r"C:\Users\Administrator\Desktop\데분개\데이터\측정소 데이터\측정소별_정보및평균.xlsx")

# 2. 좌표 컬럼을 숫자로 변환 (문자열 → float), NaN 제거
df['dmX'] = pd.to_numeric(df['dmX'], errors='coerce')  # 경도
df['dmY'] = pd.to_numeric(df['dmY'], errors='coerce')  # 위도
df = df.dropna(subset=['dmX', 'dmY'])

# 3. GeoDataFrame으로 변환 (EPSG:4326: 위경도)
geometry = [Point(x, y) for x, y in zip(df['dmY'], df['dmX'])]
gdf_station = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

# 4. 거리 단위로 buffer 계산을 위해 EPSG:5181 (UTM-K: meter 기반)로 변환
gdf_station = gdf_station.to_crs(epsg=5181)

# 5. 영향반경 정의 (단위: 미터)
radius_map = {
    '도시대기': 3000,
    '도로변대기': 300,
    '교외대기': 20000,
    '항만': 10000,
    '국가배경농도(도서)': 50000
}

# 6. 반경 컬럼 생성 및 매핑
gdf_station['radius'] = gdf_station['mangName'].map(radius_map)

# 7. 매핑 안 된 값 제거
gdf_station = gdf_station.dropna(subset=['radius'])

# 8. 영향반경 원형 buffer 생성
gdf_station['buffer'] = gdf_station.buffer(gdf_station['radius'])

# 9. 확인
print(gdf_station[['stationName', 'mangName', 'radius', 'geometry', 'buffer']].head())


  stationName    mangName  radius                       geometry  \
0         3공단        도시대기    3000  POINT (414424.576 276556.612)   
1         4공단        도시대기    3000  POINT (329488.967 294720.121)   
2         가거도  국가배경농도(도서)   50000    POINT (23901.634 66311.668)   
3         가남읍        도시대기    3000   POINT (248395.08 411541.058)   
4         가덕면        도시대기    3000  POINT (249273.224 339254.012)   

                                              buffer  
0  POLYGON ((417424.576 276556.612, 417410.13 276...  
1  POLYGON ((332488.967 294720.121, 332474.521 29...  
2  POLYGON ((73901.634 66311.668, 73660.87 61410....  
3  POLYGON ((251395.08 411541.058, 251380.634 411...  
4  POLYGON ((252273.224 339254.012, 252258.778 33...  


In [ ]:
union_buffer = gdf_station['buffer'].unary_union
outside_zones = merged.to_crs(epsg=5181)[~merged.to_crs(epsg=5181).geometry.intersects(union_buffer)]
outside_zones.to_crs(epsg=4326)  # 지도에서 보기 위해 위경도로 다시 변환

In [ ]:
outside_zones.head()

## **측정소 영향반경이 30% 이하로 겹치는 생활권만 시각화**

In [ ]:
import folium

# 1. 생활권도 거리기반 좌표계로 맞춤 (EPSG:5181)
merged_utm = merged.to_crs(epsg=5181)

# 2. 측정소 영향반경 전체 union (이미 계산된 buffer 사용)
influence_union = gdf_station['buffer'].unary_union

# 3. 생활권 별로 겹치는 비율 계산
def get_overlap_ratio(row, union_geom):
    inter_area = row.geometry.intersection(union_geom).area
    total_area = row.geometry.area
    return inter_area / total_area if total_area > 0 else 0

# 4. 겹치는 비율 컬럼 추가
merged_utm["겹침 비율"] = merged_utm.apply(
    lambda row: get_overlap_ratio(row, influence_union), axis=1
)

# 5. 50% 이하로만 겹치는 생활권 추출
outside_zones = merged_utm[merged_utm["겹침 비율"] <= 0.3]

# 6. 지도 시각화를 위해 EPSG:4326으로 변환
outside_zones_wgs84 = outside_zones.to_crs(epsg=4326)
merged_wgs84 = merged.to_crs(epsg=4326)

# 7. folium 지도 생성
m = folium.Map(location=[36.5, 127.8], zoom_start=7)

# 8. 전체 생활권 연한 회색으로 표시
folium.GeoJson(
    merged_wgs84,
    name="전체 생활권",
    style_function=lambda x: {
        "fillColor": "lightgray",
        "color": "gray",
        "weight": 0.3,
        "fillOpacity": 0.3
    }
).add_to(m)

# 9. 영향 반경과 50% 이하로만 겹치는 생활권 진한 빨간색으로 강조
folium.GeoJson(
    outside_zones_wgs84,
    name="30% 이하 영향 생활권",
    style_function=lambda x: {
        "fillColor": "red",
        "color": "darkred",
        "weight": 1,
        "fillOpacity": 0.7
    },
    tooltip=folium.GeoJsonTooltip(fields=["ID_x", "읍면동_y", "총인구수_y", "겹침 비율"])
).add_to(m)

# 10. 지도 저장
m.save("influence_below_30_percent.html")
m


## **영향반경 겹침 비율에 따른 시각화**

In [ ]:
import folium

# 1. 생활권도 거리기반 좌표계로 맞춤 (EPSG:5181)
merged_utm = merged.to_crs(epsg=5181)

# 2. 측정소 영향반경 전체 union (이미 계산된 buffer 사용)
influence_union = gdf_station['buffer'].unary_union

# 3. 겹치는 비율 계산 함수
def get_overlap_ratio(row, union_geom):
    inter_area = row.geometry.intersection(union_geom).area
    total_area = row.geometry.area
    return inter_area / total_area if total_area > 0 else 0

# 4. 비율 계산해서 열로 추가
merged_utm["겹침 비율"] = merged_utm.apply(
    lambda row: get_overlap_ratio(row, influence_union), axis=1
)

# 5. WGS84로 좌표계 변환
merged_wgs84 = merged_utm.to_crs(epsg=4326)

# 6. folium 지도 생성
m = folium.Map(location=[36.5, 127.8], zoom_start=7)

# 7. 전체 생활권 연한 회색으로 표시
folium.GeoJson(
    merged_wgs84,
    name="전체 생활권 (회색 배경)",
    style_function=lambda x: {
        "fillColor": "lightgray",
        "color": "gray",
        "weight": 0.3,
        "fillOpacity": 0.2
    }
).add_to(m)

# 8. 비율에 따라 색상 다르게 설정
def style_by_overlap(feature):
    ratio = feature["properties"]["겹침 비율"]
    if ratio <= 0.3:
        color = "red"
    elif ratio <= 0.5:
        color = "orange"
    elif ratio <= 0.7:
        color = "yellow"
    else:
        color = "green"
    return {
        "fillColor": color,
        "color": "black",
        "weight": 0.7,
        "fillOpacity": 0.6
    }

# 9. GeoJson 레이어 추가
folium.GeoJson(
    merged_wgs84,
    name="영향 반경 겹침 비율 시각화",
    style_function=style_by_overlap,
    tooltip=folium.GeoJsonTooltip(fields=["ID_x", "읍면동_y", "총인구수_y", "겹침 비율"])
).add_to(m)

# 10. 범례 수동 추가
legend_html = """
<div style="position: fixed; 
     bottom: 40px; left: 40px; width: 200px; height: 140px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white; padding: 10px;">
     <b>겹침 비율 범례</b><br>
     <i style="background:red;width:10px;height:10px;display:inline-block;"></i> 0~0.3<br>
     <i style="background:orange;width:10px;height:10px;display:inline-block;"></i> 0.3~0.5<br>
     <i style="background:yellow;width:10px;height:10px;display:inline-block;"></i> 0.5~0.7<br>
     <i style="background:green;width:10px;height:10px;display:inline-block;"></i> 0.7~1.0<br>
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

# 11. 저장
m.save("influence_overlap_4levels.html")
m


In [22]:
outside_zones.head()

,ID_x,adm_nm,adm_cd2,sgg,sido,sidonm,sggnm,adm_cd,시도_x,시군구_x,...,ID_y,시도_y,시군구_y,읍면동_y,행정기관코드_y,총인구수_y,어린이_y,노인_y,면적,겹침 비율
15,16,서울특별시 용산구 원효로2동,1117057000,11170,11,서울특별시,용산구,11030570,서울특별시;서울특별시,용산구;용산구,...,16,서울특별시;서울특별시,용산구;용산구,원효로제2동;효창동,1117057000,24022,2936,4274,1.213028e+06,0.272081
16,17,서울특별시 용산구 용문동,1117059000,11170,11,서울특별시,용산구,11030590,서울특별시,용산구,...,17,서울특별시,용산구,용문동,1117059000,11141,1172,2381,2.680860e+05,0.000000
71,72,서울특별시 중랑구 묵1동,1126062000,11260,11,서울특별시,중랑구,11070640,서울특별시,중랑구,...,72,서울특별시,중랑구,묵제1동,1126062000,33343,4384,6906,1.207708e+06,0.254932
114,115,서울특별시 도봉구 도봉1동,1132052100,11320,11,서울특별시,도봉구,11100640,서울특별시,도봉구,...,115,서울특별시,도봉구,도봉제1동,1132052100,19615,1460,6479,8.508462e+06,0.290185
123,124,서울특별시 노원구 월계1동,1135056000,11350,11,서울특별시,노원구,11110510,서울특별시,노원구,...,124,서울특별시,노원구,월계1동,1135056000,20342,2290,4660,1.178816e+06,0.150268


## **영향 밖 생활권 정보 csv 저장**

In [23]:
# EPSG:4326으로 변환 (한 번만 하면 생략 가능)
outside_zones_wgs84 = outside_zones.to_crs(epsg=4326)

# 내가 저장하고 싶은 열만 골라서 지정
selected_cols = ['ID_x', '시도_y', '시군구_y', '읍면동_y', '행정기관코드_y', '총인구수_y', '어린이_y', '노인_y',
                 '겹침 비율']

# 저장 경로 지정
save_path = r"C:\Users\Administrator\Desktop\데분개\영향밖_생활권_정보.csv"

# 선택한 열만 저장
outside_zones_wgs84[selected_cols].to_csv(save_path, index=False, encoding='utf-8-sig')

print(f"✔ 선택한 열만 포함된 CSV 파일이 저장되었습니다:\n{save_path}")


✔ 선택한 열만 포함된 CSV 파일이 저장되었습니다:
C:\Users\Administrator\Desktop\데분개\영향밖_생활권_정보.csv
